# Trignometric Pose Corrector
## Planks

### importing libraries

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math

### Initiainting Medaipipe

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

### Function to calculate joint angles using 3 Coordinates

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle


## For uisng Local Video Input

In [5]:
cap = cv2.VideoCapture('msc.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(203,17,17), thickness=2, circle_radius=2) 
                                     )  

    landmarks =  result.pose_landmarks.landmark
    right_side_confidence = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_HIP].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility
    left_side_confidence = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].visibility + landmarks[mp_pose.PoseLandmark.LEFT_HIP].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].visibility 
    if right_side_confidence > left_side_confidence: 

        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        #calc angle for shoulder-elbow-wrist
        #print(shoulder,elbow,wrist)


        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        #calc angle for shoulder-hip-knee

        ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
        #calculate ANGLE FOR knee-ankle-foot-index
        
    if right_side_confidence < left_side_confidence: 
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        #calc angle for shoulder-elbow-wrist
        #print(shoulder,elbow,wrist)
        
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        #calc angle for shoulder-hip-knee

        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
#         calculate ANGLE FOR knee-ankle-foot-index
    
    angle_sew = calculate_angle(shoulder,elbow,wrist)
    angle_shk = calculate_angle(shoulder,hip,knee)
    angle_kai = calculate_angle(knee,ankle,foot_index)
    angle_hse = calculate_angle(hip,shoulder,elbow)
    #angle for hip shoulder elbow
    angle_hse = calculate_angle(hip,shoulder,elbow)
    angle_hse = 180-angle_hse
#    angle_sew = 180-angle_sew
    angle_kai = 180-angle_kai
  #  angle_shk = 180-angle_shk
    
    print(angle_hse,angle_kai,angle_shk,angle_sew)
    org = (20, 20)
    org1 = (20, 45)
    org2 = (20, 60)
    org3 = (20, 80)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontscale = 0.7
    color = (12, 15, 15)
    thickness = 2
    lineType = cv2.LINE_AA
    correction  = False
    
    if(angle_hse < 65):
        correction = True
        cv2.putText(frame,' Correct your Shoulder angle by brinigng your arms closer to your body',org,font,fontscale,color,thickness,lineType)
    if(angle_hse > 95):
        correction = True
        cv2.putText(frame,' Correct your Shoulder angle by moving your arms farther from your body',org,font,fontscale,color,thickness,lineType)  
    if(angle_sew<45):
        correction = True
        cv2.putText(image,'eblow angle too small try pulling your body sligthly upwards',org2,font,fontscale,color,thickness,lineType)
    if(angle_sew > 90):
        correction = True
        cv2.putText(frame,' eblow angle too large try  lowering your body ',org2,font,fontscale,color,thickness,lineType)
    if(angle_shk < 135):
        correction = True
        cv2.putText(frame,'tigthen your core',org1,font,fontscale,color,thickness,lineType) 

    if(angle_shk > 170):
        correction = True
        cv2.putText(frame,'MOVE YOUR HIPS SLIGHTLY UPWARDS',org1,font,fontscale,color,thickness,lineType) 

    if(angle_kai < 70):
        correction = True
        cv2.putText(frame,'Move your toes outwards',org3,font,fontscale,color,thickness,lineType) 
    if(angle_kai > 120):
        correction = True
        cv2.putText(frame,' move your toes inwards',org3,font,fontscale,color,thickness,lineType) 
    if correction:
        cv2.putText(frame,' Incorrect Posture',(450,200),font,fontscale+0.4,color,thickness,lineType)
    else:
        cv2.putText(frame,' Correct Posture',(450,250),font,fontscale+0.4,color,thickness,lineType)
        
#     cv2.resizeWindow('Video with Pose Analysis', 800, 600)  # Resize the window to your desired size
    cv2.imshow('Video with Pose Analysis', frame)
     # Start the timer for each frame
    start_time = time.time()

    # Add the timer on the frame
    elapsed_time = round(time.time() - start_time, 2)
    cv2.putText(frame, f"Elapsed Time: {elapsed_time} seconds", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (123, 123, 255), 2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
#cv2.imwrite('images/result_image.png',frame)


75.90394139012949 76.4363538350566 140.0080178182985 67.89111562843736
77.91896894302448 75.57566506426392 142.9074065056205 70.11819322343155
77.4767713269921 75.94126668345066 141.8154610371686 71.11364135723277
76.77554056894715 75.11404113075798 141.18888201513775 72.47989233187505
76.65188103421382 76.69138219594419 140.6016868138048 72.10336874780913
77.13733143168047 76.65850454403244 142.2593293857547 73.96634091033371
76.85539775661039 76.02724385445435 140.91471112649486 74.4797242284197
76.72797163585372 76.85762533139399 140.85410770245835 74.75807848098682
76.33737558538604 76.94021560846875 139.21578363652435 75.44151815183793
76.85266016341515 75.66114231278041 139.51227704543695 76.17511821834067
76.8036160744569 75.4644754874825 138.80363120096357 75.45104538145098
77.26331462024336 76.3628608329737 139.4334364443963 75.70390574244098
77.42112917666117 76.02224873162999 139.70981564542208 76.06972262232719
79.19939706912565 73.57829187705687 139.8343736111691 80.757122

## For Live Video Input

In [6]:
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit()

# Set the width and height of the frame
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Capture frames from the camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    if result.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(203, 17, 17), thickness=2, circle_radius=2)
        )

        landmarks = result.pose_landmarks.landmark

        # Confidence calculations and angle measurements
        try:
            right_side_confidence = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + \
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + \
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            left_side_confidence = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + \
                                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + \
                                   landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility

            if right_side_confidence > left_side_confidence:
                shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

            else:
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

            angle_sew = calculate_angle(shoulder, elbow, wrist)
            angle_shk = calculate_angle(shoulder, hip, knee)
            angle_kai = calculate_angle(knee, ankle, foot_index)
            angle_hse = calculate_angle(hip, shoulder, elbow)
            angle_hse = 180 - angle_hse
            angle_kai = 180 - angle_kai

            print(angle_hse, angle_kai, angle_shk, angle_sew)

            org = (20, 20)
            org1 = (20, 45)
            org2 = (20, 60)
            org3 = (20, 80)
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontscale = 0.7
            color = (12, 15, 15)
            thickness = 2
            lineType = cv2.LINE_AA
            correction = False

            if angle_hse < 65:
                correction = True
                cv2.putText(frame, 'Correct your Shoulder angle by bringing your arms closer to your body', org, font, fontscale, color, thickness, lineType)
            if angle_hse > 95:
                correction = True
                cv2.putText(frame, 'Correct your Shoulder angle by moving your arms farther from your body', org, font, fontscale, color, thickness, lineType)
            if angle_sew < 45:
                correction = True
                cv2.putText(frame, 'Elbow angle too small, try pulling your body slightly upwards', org2, font, fontscale, color, thickness, lineType)
            if angle_sew > 90:
                correction = True
                cv2.putText(frame, 'Elbow angle too large, try lowering your body', org2, font, fontscale, color, thickness, lineType)
            if angle_shk < 135:
                correction = True
                cv2.putText(frame, 'Tighten your core', org1, font, fontscale, color, thickness, lineType)
            if angle_shk > 170:
                correction = True
                cv2.putText(frame, 'Move your hips slightly upwards', org1, font, fontscale, color, thickness, lineType)
            if angle_kai < 70:
                correction = True
                cv2.putText(frame, 'Move your toes outwards', org3, font, fontscale, color, thickness, lineType)
            if angle_kai > 120:
                correction = True
                cv2.putText(frame, 'Move your toes inwards', org3, font, fontscale, color, thickness, lineType)

            if correction:
                cv2.putText(frame, 'Incorrect Posture', (450, 200), font, fontscale + 0.4, color, thickness, lineType)
            else:
                cv2.putText(frame, 'Correct Posture', (450, 250), font, fontscale + 0.4, color, thickness, lineType)

        except IndexError:
            print("Error: Not all landmarks are detected.")

    cv2.imshow('Video with Pose Analysis', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

15.698466599622662 56.444277344754525 29.25094756363259 175.5206972600017
160.63945452633317 19.780817418661485 174.59821518227713 166.08867746844118
162.6733910070208 17.44631169595425 173.8109150378168 166.71388577345465
160.27068867726052 16.000719518424575 174.02879841411303 160.503958952603
159.3398067277182 15.358207971641235 173.9858765479378 158.40199190784432
160.11560691609114 15.708125107339782 174.65356917185386 157.77943682770507
160.05324946064258 16.081192493641993 174.66963777120176 158.17123657589366
160.98527362048065 13.307127257634818 174.443278663576 159.72925977473898
161.1257048700153 13.58843995070228 174.33938096689494 160.44295699148367
161.01711568885523 14.329958976863082 174.3127517708818 160.4054394186213
160.9137406730119 13.59832138338362 174.26024576516105 160.2085384766699
161.6853351946845 16.48327992977721 174.07269969080096 159.94169350036108
161.4557119471066 17.753067529678788 174.38097485959656 160.45634403012735
161.18703733474044 16.81811327590

165.3835350109432 12.62553097343536 174.9208838310131 168.82560008578707
164.8516753990254 12.237444364751099 174.9281619030285 169.2315623774633
168.16486752277194 11.934385130613464 175.20915497332106 170.6164141132877
168.76458305128835 12.044937856130588 175.13111478393873 170.5954807836292
169.08659247142663 12.08795117742497 175.14382792651162 170.32328107893474
167.7878981719768 12.070346271667944 174.6499764533134 171.60314508894996
167.0265355666325 11.965424922088857 174.3977747807218 171.66360841224113
166.49012514419496 11.866778654240221 174.27602347409544 170.991903206483
166.32905961641208 12.418139119439218 174.22019380684833 170.45694919975838
168.76975003431625 12.994338093116653 174.07327973759385 172.382386608138
169.12470004792874 13.91869948992698 174.43030281392845 169.15814482009176
168.74279419105935 13.51988922294106 174.6229811242948 168.6597938370714
168.30812263659374 11.594416529079211 175.08105449487186 23.39691702461315
168.6942413667789 12.2141517926266

166.4104229298729 12.48746171457384 174.9967965742594 172.27570912217485
166.60039148509065 12.596353915810596 174.92805154418184 172.59748629481626
166.59165322296377 12.64796832564457 174.83876164282722 172.41640648436083
166.8051334824722 11.962233952691065 174.96127742992763 175.53825532214648
167.05351250930966 11.56201446536133 175.21597250004996 176.93120348631152
167.21595300384882 11.513304910170888 175.35343301327933 177.42148184176878
168.9194146825839 12.046943108540802 175.50171249380628 176.65301061994592
167.47855773946353 12.23753057105003 175.53999051206938 174.3798680181254
167.54918792698356 12.654214755949937 175.37446085354895 172.22462387631006
167.74453946688902 12.83967309972536 175.35408076395413 170.59364150290511
143.69591218715965 12.497694581717525 175.35108661063134 157.60764782422217
152.8790170536542 12.555777204972912 175.38903195799796 167.8201068006241
157.20444038743173 12.695940074109615 175.45875963945517 173.16897595340188
167.03115200689746 12.92

## For a Single Frame

In [ ]:
frame =  cv2.imread('images/plankm.jpg')
rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
result = pose.process(rgb_frame)

mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(203,17,17), thickness=2, circle_radius=2) 
                                 )

landmarks =  result.pose_landmarks.landmark
right_side_confidence = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_HIP].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility
left_side_confidence = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].visibility + landmarks[mp_pose.PoseLandmark.LEFT_HIP].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].visibility 
if right_side_confidence > left_side_confidence: 

    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    #calc angle for shoulder-elbow-wrist
    #print(shoulder,elbow,wrist)


    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    #calc angle for shoulder-hip-knee

    ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
#calculate ANGLE FOR knee-ankle-foot-index
if right_side_confidence < left_side_confidence: 
    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    #calc angle for shoulder-elbow-wrist
    #print(shoulder,elbow,wrist)

    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    #calc angle for shoulder-hip-knee

    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
#     calculate ANGLE FOR knee-ankle-foot-index

angle_sew = calculate_angle(shoulder,elbow,wrist)
angle_shk = calculate_angle(shoulder,hip,knee)
angle_kai = calculate_angle(knee,ankle,foot_index)
angle_hse = calculate_angle(hip,shoulder,elbow)
#angle for hip shoulder elbow
angle_hse = calculate_angle(hip,shoulder,elbow)
angle_hse = 180-angle_hse
#    angle_sew = 180-angle_sew
angle_kai = 180-angle_kai
#  angle_shk = 180-angle_shk

print(angle_hse,angle_kai,angle_shk,angle_sew)
org = (20, 20)
org1 = (400, 100)
org2 = (20, 60)
org3 = (20, 80)
font = cv2.FONT_HERSHEY_SIMPLEX
fontscale = 0.7
color = (12, 15, 15)
thickness = 2
lineType = cv2.LINE_AA
correction  = False


if(angle_hse < 65):
    correction = True
    cv2.putText(frame,' Correct your Shoulder angle by brinigng your arms closer to your body',org,font,fontscale,color,thickness,lineType)
if(angle_hse > 95):
    correction = True
    cv2.putText(frame,' Correct your Shoulder angle by moving your arms farther from your body',org,font,fontscale,color,thickness,lineType)  
if(angle_sew<45):
    correction = True
    cv2.putText(image,'eblow angle too small try pulling your body sligthly upwards',org2,font,fontscale,color,thickness,lineType)
if(angle_sew > 90):
    correction = True
    cv2.putText(frame,' eblow angle too large try  lowering your body ',org2,font,fontscale,color,thickness,lineType)
if(angle_shk < 135):
    correction = True
    cv2.putText(frame,'tigthen your core',org1,font,fontscale,color,thickness,lineType) 

if(angle_shk > 170):
    correction = True
    cv2.putText(frame,'MOVE YOUR HIPS SLIGHTLY UPWARDS',org1,font,fontscale,color,thickness,lineType) 

if(angle_kai < 70):
    correction = True
    cv2.putText(frame,'Move your toes outwards',org3,font,fontscale,color,thickness,lineType) 
if(angle_kai > 120):
    correction = True
    cv2.putText(frame,' move your toes inwards',org3,font,fontscale,color,thickness,lineType) 
if correction:
    cv2.putText(frame,' Incorrect Posture',(450,150),font,fontscale+0.4,color,thickness,lineType)
else:
    cv2.putText(frame,' Correct Posture',(450,250),font,fontscale+0.4,color,thickness,lineType)

cv2.imshow('plank',frame)
cv2.waitKey(0)  # Wait for any key to be pressed
cv2.destroyAllWindows()